## 프롬프트

In [1]:
from dotenv import load_dotenv
import os, sys

load_dotenv()
sys.path.append(os.path.abspath(".."))

In [ ]:
from app import langsmith

langsmith("Prompt", set_enable=False)

LangSmith 추적을 하지 않습니다.


### 프롬프트 템플릿 만들기
구조
- 지시사항 (Instruction)
- 사용자가 입력한 질문 (Question)
- 검색된 정보인 문맥 (Context)

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI()

#### ① `from_template` 메서드 이용

In [4]:
# 변수를 포함한 프롬프트 템플릿 만들기
template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [5]:
prompt.format(country="대한민국")

'대한민국의 수도는 어디인가요?'

In [6]:
chain = prompt | llm
chain.invoke("대한민국").content

'대한민국의 수도는 서울이며, 서울특별시는 대한민국의 정치·경제·문화의 중심지로서 수도인 서울특별시는 대한민국의 70%가 넘는 종다수의 공공기관과 기업이 본점을 뒀으며 중앙정부와 국회가 위치하고 있다.'

#### ② `PromptTemplate` 객체에 `input_variables` 리스트를 명시적으로 전달

In [7]:
template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["country"],
)

prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [8]:
prompt.format(country="대한민국")

'대한민국의 수도는 어디인가요?'

In [9]:
# 일부 변수를 미리 확정지을 때
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={"country2": "일본"},
)

prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '일본'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [10]:
prompt.format(country1="대한민국")

'대한민국과 일본의 수도는 각각 어디인가요?'

In [11]:
# 일부 변수 미리 채우기
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [12]:
prompt_partial.format(country1="대한민국")

'대한민국과 캐나다의 수도는 각각 어디인가요?'

In [13]:
chain = prompt_partial | llm
chain.invoke("대한민국").content

'대한민국의 수도는 서울이고, 캐나다의 수도는 오타와입니다.'

In [14]:
# partial()로 미리 채운 변수도 invoke()에 딕셔너리 형태로 전달 가능
chain.invoke({"country1": "대한민국", "country2": "호주"}).content

'대한민국의 수도는 서울이며 호주의 수도는 캔버라입니다.'

### 부분 변수 활용하기

In [15]:
from datetime import datetime

datetime.now().strftime("%B %d") # 오늘 날짜 출력

'October 10'

In [16]:
def get_today(): # 날짜 반환 함수
    return datetime.now().strftime("%B %d")

In [17]:
prompt = PromptTemplate(
    template="오늘 날짜는 {today}입니다. 오늘이 생일인 유명인 {n}명을 나열해주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={"today": get_today},
)

In [18]:
prompt.format(n=3)

'오늘 날짜는 October 10입니다. 오늘이 생일인 유명인 3명을 나열해주세요. 생년월일을 표기해주세요.'

In [19]:
chain = prompt | llm
print(chain.invoke(3).content)

1. 배우 이민호 (1987년 6월 22일)
2. 가수 신용재 (1983년 10월 11일)
3. 배우 배용준 (1989년 10월 15일)


In [20]:
# 미리 설정해둔 변수에 다른 값 전달하기
print(chain.invoke({"today": "Jan 02", "n": 3}).content)

1. 김태희 - 1980년 3월 29일
2. 김소현 - 1996년 1월 7일
3. 스티브 조브스 - 1955년 2월 24일


### YAML 파일로부터 프롬프트 템플릿 로드하기

In [22]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("../prompts/fruit_color.yaml", encoding="utf-8") # YAML 파일로부터 프롬프트 템플릿 로드하기
prompt

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')

In [23]:
prompt.format(fruit="사과")

'사과의 색깔이 뭐야?'

In [27]:
prompt2 = load_prompt("../prompts/capital.yaml")
print("프롬프트:", prompt2, end="\n\n")

print(prompt2.format(country="대한민국"))

프롬프트: input_variables=['country'] input_types={} partial_variables={} template='{country}의 수도에 대해서 알려주세요.\n수도의 특징을 다음의 양식에 맞게 정리해 주세요.\n300자 내외로 작성해 주세요.\n한글로 작성해 주세요.\n----\n[양식]\n1. 면적\n2. 인구\n3. 역사적 장소\n4. 특산품\n\n#Answer:\n'

대한민국의 수도에 대해서 알려주세요.
수도의 특징을 다음의 양식에 맞게 정리해 주세요.
300자 내외로 작성해 주세요.
한글로 작성해 주세요.
----
[양식]
1. 면적
2. 인구
3. 역사적 장소
4. 특산품

#Answer:



In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessageChunk
from app import stream_response

In [29]:
chain = prompt2 | ChatOpenAI(model="gpt-4o", temperature=0) | StrOutputParser()

answer = chain.stream({"country": "대한민국"})
stream_response(answer)

1. 면적: 대한민국의 수도인 서울특별시는 약 605.21㎢의 면적을 가지고 있습니다. 이는 전국에서 가장 큰 도시 중 하나로, 다양한 문화와 경제 활동이 이루어지는 중심지입니다.

2. 인구: 서울의 인구는 약 950만 명으로, 대한민국에서 가장 인구가 많은 도시입니다. 다양한 인종과 문화가 공존하며, 국제적인 도시로서의 면모를 갖추고 있습니다.

3. 역사적 장소: 서울에는 경복궁, 창덕궁, 덕수궁 등 조선시대의 궁궐들이 잘 보존되어 있으며, 인사동과 북촌 한옥마을 같은 전통 문화 지역도 있습니다. 또한, 서울 성곽과 같은 유적지도 있어 역사적 가치가 높습니다.

4. 특산품: 서울은 전통적으로 한복, 한지, 도자기 등이 유명하며, 현대에는 다양한 패션과 뷰티 제품이 특산품으로 자리 잡고 있습니다. 또한, 서울의 다양한 음식 문화는 김치, 불고기, 비빔밥 등으로 대표됩니다.

### `ChatPromptTemplate`
일반 프롬프트 템플릿 : 일방적인 지시 사항 전달    
↔ ChatPromptTemplate : 사용자가 대화를 주고받는 형식, 더 유연하고 자연스러운 결과 출력

`(role, message)` 형태의 튜플 형식으로 구성    
- system 
  - 시스템 설정 메시지
  - 전역 설정과 관련된 프롬프트, 대화 전체에 적용되는 설정
  - AI의 역할이나 페르소나 지정
- human : 사용자가 입력하는 메시지
- ai : AI의 응답 메시지

In [30]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?'), additional_kwargs={})])

In [31]:
# Human이라는 역할이 자동으로 지정됨
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

In [32]:
# 메시지 형식으로 프롬프트 템플릿 만들기
chat_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name}입니다."),
    ("human", "반가워요!"),
    ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
    ("human", "{user_input}")
])

In [33]:
# 프롬프트 생성
messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 테디입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]

In [34]:
llm = ChatOpenAI()
llm.invoke(messages).content

'제 이름은 테디입니다. 어떻게 도와드릴까요?'

In [35]:
chain = chat_template | llm
chain.invoke({"name": "테디", "user_input": "당신의 이름은 무엇입니까?"}).content

'제 이름은 테디입니다. 어떻게 도와드릴까요?'

### `MessagesPlaceHolder`
- 나중에 채워질 메시지를 채우기 위해 임시로 확보한 자리
- 대화 내용을 기록할 때 자주 사용 : 대화 진행 중에 `MessagesPlaceHolder`에 저장해 두었다가 나중에 대화 기록을 분석하거나 다른 질문에 활용

In [36]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# human 메시지에서 대화 내용을 {word_count} 단어로 요약
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다."),
    MessagesPlaceholder(variable_name="conversation"),
    ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
])
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

In [37]:
formatted_chat_prompt = chat_prompt.format_prompt(
    word_count=10,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁드립니다."),
    ]
)
print(formatted_chat_prompt)

'''
System : '당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.'
Human : '안녕하세요! 저는 오늘 새로 입사한 테디입니다. 만나서 반갑습니다.'
AI : '반가워요! 앞으로 잘 부탁드립니다.'
Human : '지금까지의 대화를 10 단어로 요약합니다.'
'''

messages=[SystemMessage(content='당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요! 저는 오늘 새로 입사한 테디입니다. 만나서 반갑습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='반가워요! 앞으로 잘 부탁드립니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='지금까지의 대화를 10 단어로 요약합니다.', additional_kwargs={}, response_metadata={})]


"\nSystem : '당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.'\nHuman : '안녕하세요! 저는 오늘 새로 입사한 테디입니다. 만나서 반갑습니다.'\nAI : '반가워요! 앞으로 잘 부탁드립니다.'\nHuman : '지금까지의 대화를 10 단어로 요약합니다.'\n"

In [38]:
# 체인 생성
llm = ChatOpenAI(model="gpt-4o")
chain = chat_prompt | llm | StrOutputParser()
chain.invoke({
    "word_count": 10,
    "conversation": [
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁드립니다."),
    ]
})

'입사, 인사, 테디, 새로, 반가움, 대화, 안녕하세요, 만남, 부탁드립니다.'

### 퓨샷 프롬프트
- 퓨샷 기법 (`FewShotPromptTemplate`) : 예시를 제공하여 답변 형식을 명시
- 원샷 : 하나의 답변 예시 제공
- 퓨샷 : 두 개 이상의 예시 제공

In [39]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(temperature=0, model="gpt-4.1")

In [40]:
examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

In [41]:
example_prompt = PromptTemplate.from_template(
    "Question: {question}\nAnswer: {answer}\n"
)

print(example_prompt.format(**examples[0]))

Question: 스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer: 이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인




In [42]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: \n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 빌게이츠의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

Question: 스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer: 이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인



Question: 네이버의 창립자는 언제 태어났나요?
Answer: 이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일



Question: 율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?
Answer: 이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군



Question: 올드보이와 기생충의 감독이 같은 나라 출신인가요?
Answer: 이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예



Question: 
Google이 창립된 연도에 빌게이츠의 나이는 몇 살인가요?
Answer:


In [43]:
answer = llm.stream(final_prompt)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google이 창립된 연도는 언제인가요?  
중간 답변: Google은 1998년에 창립되었습니다.  
추가 질문: 빌 게이츠는 언제 태어났나요?  
중간 답변: 빌 게이츠는 1955년 10월 28일에 태어났습니다.  
추가 질문: 1998년에 빌 게이츠의 나이는 몇 살이었나요?  
중간 답변: 1998년 - 1955년 = 43년. 1998년 10월 28일 이전에는 42세, 이후에는 43세입니다.  
최종 답변은: 42세 또는 43세였습니다.

In [44]:
# 체인 생성
chain = prompt | llm | StrOutputParser()

answer = chain.stream(
    {"question": "Google이 창립된 연도에 빌게이츠의 나이는 몇 살인가요?"}
)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google이 창립된 연도는 언제인가요?  
중간 답변: Google은 1998년에 창립되었습니다.  
추가 질문: 빌 게이츠는 언제 태어났나요?  
중간 답변: 빌 게이츠는 1955년 10월 28일에 태어났습니다.  
추가 질문: 1998년에 빌 게이츠의 나이는 몇 살이었나요?  
중간 답변: 1998년에서 1955년을 빼면 43이므로, 1998년 10월 28일 이전에는 42세, 이후에는 43세입니다.  
최종 답변은: 42세 또는 43세였습니다. (1998년 10월 28일 이전에는 42세, 이후에는 43세)

### 예제 선택기
질문과 유사한 예시만 서택해 프롬프트에 넣는 방식
- `SemanticSimilarityExampleSelector` : 입력된 질문과 의미가 가장 유사한 예시 선택 
- `MaxMarginalRelevanceExampleSelector` : 입력된 질문과 가장 유사하면서도 중복되지 않는 예시 선택

In [45]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector, MaxMarginalRelevanceExampleSelector
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

chroma = Chroma("example_selector", OpenAIEmbeddings())

In [46]:
'''
임베딩 : 문장 간 유사성을 수학적으로 계산하기 위한 방법
문장을 수치로 변환해 거리를 계산
두 문장이 유사할수록 임베딩된 값이 가까워짐
'''

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,               # 선택 가능한 예시 목록
    OpenAIEmbeddings(),     # 임베딩 클래스
    Chroma,                 # 벡터 스토어
    k=1                     # 생성할 예시 수
)

In [47]:
selected_examples = example_selector.select_examples({"question": question})

question = "Google이 창립된 연도에 빌게이츠의 나이는 몇 살인가요?"
print(f"입력에 가장 유사한 예시: \n{question}\n")

for example in selected_examples:
    print(f'question: \n{example["question"]}')
    print(f'answer: \n{example["answer"]}')

입력에 가장 유사한 예시: 
Google이 창립된 연도에 빌게이츠의 나이는 몇 살인가요?

question: 
네이버의 창립자는 언제 태어났나요?
answer: 
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일



In [48]:
# 프롬프트 예시를 구성하고 LLM과 연결해 질문에 대한 답변 출력
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question: \n{question}\nAnswer: ",
    input_variables=["question"],
)

chain = prompt | llm

answer = chain.stream({"question": "Google이 창립된 연도에 빌게이츠의 나이는 몇 살인가요?"})
stream_response(answer)

Google은 1998년에 창립되었습니다.  
빌 게이츠(Bill Gates)는 1955년 10월 28일에 태어났습니다.

1998년 - 1955년 = 43

따라서, Google이 창립된 1998년에 빌 게이츠의 나이는 43세였습니다.

### `FewShotChatMessagePromptTemplate`
`FewShotPromptTemplate`에서 ChatMessage 추가

In [49]:
examples = [
    {
        "instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
        "input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
        "answer": """
회의록: XYZ 회사 마케팅 전략 회의
일시: 2023년 12월 25일
장소: XYZ 회사 회의실
참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)

1. 개회
   - 회의는 김수진 팀장의 개회사로 시작됨.
   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.

2. 시장 동향 개요 (김수진)
   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.
   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.

3. 디지털 마케팅 전략 (박지민)
   - 박지민은 디지털 마케팅 전략에 대해 발표.
   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.

4. 소셜 미디어 캠페인 (이준호)
   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.
   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.

5. 종합 논의
   - 팀원들 간의 아이디어 공유 및 토론.
   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.

6. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 박지민 담당.
""",
    },
    {
        "instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
        "input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
        "answer": """
문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서

- 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.
- 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.
- 전략: 지속 가능한 도시 개발을 달성하기 위한 다양한 전략 제시. 이에는 친환경 건축, 대중교통 개선, 에너지 효율성 증대, 지역사회 참여 강화 등이 포함됨.
- 사례 연구: 전 세계 여러 도시의 성공적인 지속 가능한 개발 사례를 소개. 예를 들어, 덴마크의 코펜하겐, 일본의 요코하마 등의 사례를 통해 실현 가능한 전략들을 설명.
- 교훈: 이러한 사례들에서 얻은 주요 교훈을 요약. 강조된 교훈에는 다각적 접근의 중요성, 지역사회와의 협력, 장기적 계획의 필요성 등이 포함됨.

이 보고서는 지속 가능한 도시 개발이 어떻게 현실적이고 효과적인 형태로 이루어질 수 있는지에 대한 심도 있는 분석을 제공합니다.
""",
    },
    {
        "instruction": "당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요",
        "input": "우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.",
        "answer": "본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.",
    },
]

In [50]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

In [51]:
chroma = Chroma("fewshot_chat", OpenAIEmbeddings())

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{instruction}:\n{input}"),
    ("ai", "{answer}")
])

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, OpenAIEmbeddings(), chroma, k=1
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt
)

In [52]:
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

example_selector.select_examples(question)

[{'input': '2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.',
  'instruction': '당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요',
  'answer': '\n회의록: XYZ 회사 마케팅 전략 회의\n일시: 2023년 12월 25일\n장소: XYZ 회사 회의실\n참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)\n\n1. 개회\n   - 회의는 김수진 팀장의 개회사로 시작됨.\n   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.\n\n2. 시장 동향 개요 (김수진)\n   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.\n   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.\n\n3. 디지털 마케팅 전략 (박지민)\n   - 박지민은 디지털 마케팅 전략에 대해 발표.\n   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.\n\n4. 소셜 미디어 캠페인 (이준호)\n   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.\n   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.\n\n5. 종합 논의\n   - 팀원들 간의 아이디어 공유 및 토론.\n   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.\n\n6. 마무리\n   - 다음 회의 날짜 및 시간 확정.\n   - 회의록 정리 및 배포는 박지민 담당.\n'}]

In [53]:
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    few_shot_prompt,
    ("human", "{instruction}\n{input}"),
])

In [54]:
chain = final_prompt | llm

answer = chain.stream(question)
stream_response(answer)

회의록: ABC 기술 회사 제품 개발팀 주간 진행 상황 회의

일시: 2023년 12월 26일  
장소: ABC 기술 회사 회의실  
참석자: 최현수(프로젝트 매니저), 황지연(주요 개발자), 김태영(UI/UX 디자이너)

1. 개회
- 최현수 프로젝트 매니저가 회의를 시작하며 회의 목적(프로젝트 진행 상황 검토 및 다가오는 마일스톤 계획 수립)을 안내함.

2. 각 팀원별 진행 상황 보고
- 황지연(주요 개발자):  
  · 백엔드 서버 연동 작업 1차 완료  
  · 주요 기능(로그인, 데이터 동기화) 개발 진행 상황 공유  
  · 남은 이슈(알림 기능 오류) 및 해결 방안 논의

- 김태영(UI/UX 디자이너):  
  · 메인 화면 및 내비게이션 UI 시안 완성  
  · 사용자 피드백 반영을 위한 추가 디자인 수정 계획 보고  
  · 개발팀과의 협업 일정 조율 필요성 언급

3. 다가오는 마일스톤 계획
- 최현수:  
  · 다음 주까지 알림 기능 완성 및 테스트 목표 설정  
  · UI 최종 시안 확정 및 개발팀 전달 일정 조율  
  · 전체 일정표 재점검 및 위험 요소 사전 파악 요청

4. 종합 논의 및 다음 주 목표 설정
- 각 팀원별로 다음 주까지의 구체적 목표 재확인  
- 진행 상황 공유 및 이슈 발생 시 즉각 보고하기로 합의

5. 마무리
- 다음 회의 일정: 2024년 1월 2일(화) 오후 2시  
- 회의록 작성 및 배포: 최현수

회의 종료: 오후 3시 40분

### 목적에 맞는 예제 선택기

In [ ]:
from app import CustomExampleSelector

custom_selector = CustomExampleSelector(examples, OpenAIEmbeddings())

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{instruction}:\n{input}"),
    ("ai", "{answer}")
])

custom_fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=custom_selector,
    example_prompt=example_prompt
    
)

custom_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    few_shot_prompt,
    ("human", "{instruction}:\n{input}"),
])

In [57]:
chain = custom_prompt | llm

question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

stream_response(chain.stream(question))

회의록: ABC 기술 회사 제품 개발팀 주간 진행 상황 회의

일시: 2023년 12월 26일  
장소: ABC 기술 회사 회의실  
참석자: 최현수(프로젝트 매니저), 황지연(주요 개발자), 김태영(UI/UX 디자이너)

1. 개회
- 최현수 프로젝트 매니저가 회의를 시작하며 회의 목적(프로젝트 진행 상황 검토 및 마일스톤 계획 수립)을 안내함.

2. 각 팀원별 진행 상황 보고
- 황지연(주요 개발자):  
  - 백엔드 서버 연동 및 주요 기능 구현 현황 보고  
  - 현재까지 발견된 주요 이슈 및 해결 방안 공유

- 김태영(UI/UX 디자이너):  
  - 메인 화면 및 사용자 흐름 설계 진행 상황 설명  
  - 사용자 피드백 반영 계획 및 추가 디자인 작업 일정 안내

3. 마일스톤 및 향후 계획 논의
- 다가오는 마일스톤(알파 버전 개발 완료)에 대한 일정 재확인
- 각 작업별 우선순위 및 역할 분담 재조정
- 다음 주까지의 목표 설정:  
  - 개발팀: 핵심 기능 구현 마무리  
  - 디자인팀: 추가 화면 디자인 및 프로토타입 완성

4. 기타 논의 사항
- 협업 툴 활용 방안 및 커뮤니케이션 강화 방안 논의

5. 마무리
- 다음 회의 일정: 2024년 1월 2일(예정)
- 회의록 작성 및 배포 담당: 김태영

회의 종료

### LangChain Hub에서 프롬프트 공유하기

[LangChain Hub 페이지](https://smith.langchain.com/hub)

In [ ]:
from langchain import hub

# LangChain Hub에서 프롬프트 불러오기 
# pull() 메서드 안에 프롬프트 ID 입력
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
# 프롬프트 업로드

hub.push("LangChain ID/프롬프트 저장소 이름", prompt)